Try and do summarization of CFR sections

In [32]:
import pandas as pd
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize


['C:/Users/Lilly/Documents/GitHub/CFR-Parsing/nltk']


In [ ]:
# if you are trying to use this, use the line below
# nltk.download('punkt', download_dir='../nltk')

# i had problems and had to force the directory and path for nltk. don't use this!
# nltk.data.path = [] 
# nltk.data.path.append("C:/Users/Lilly/Documents/GitHub/CFR-Parsing/nltk")

In [33]:
# load the dataframe 
title_number = "12"
df = pd.read_parquet(f"../dataframe/{title_number}.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117306 entries, 0 to 117305
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   p_id          117306 non-null  object
 1   text          117306 non-null  object
 2   child_ids     117306 non-null  object
 3   cfr_links     117306 non-null  object
 4   other_links   117306 non-null  object
 5   link_targets  117306 non-null  object
dtypes: object(6)
memory usage: 5.4+ MB


In [34]:
reg_z_id = '1026.20'
reg_z_df = df[df['p_id'].str.startswith(reg_z_id)].drop_duplicates(subset='p_id')
reg_z_text_list = list(reg_z_df['text'])
print("Number of paragraphs:", len(reg_z_text_list))
reg_z_text = "\n".join(reg_z_text_list)
reg_z_text[:1000]


Number of paragraphs: 95


"(a)Refinancings.A refinancing occurs when an existing obligation that was subject to this subpart is satisfied and replaced by a new obligation undertaken by the same consumer. A refinancing is a new transaction requiring new disclosures to the consumer. The new finance charge shall include any unearned portion of the old finance charge that is not credited to the existing obligation. The following shall not be treated as a refinancing:\n(1)A renewal of a single payment obligation with no change in the original terms.\n(2)A reduction in the annual percentage rate with a corresponding change in the payment schedule.\n(3)An agreement involving a court proceeding.\n(4)A change in the payment schedule or a change in collateral requirements as a result of the consumer's default or delinquency, unless the rate is increased, or the new amount financed exceeds the unpaid balance plus earned finance charge and premiums for continuation of insurance of the types described in§ 1026.4(d).\n(5)The

In [35]:
max_length = 50 # length of the summaries

# Initialize the summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Split the text into sentences
sentences = sent_tokenize(reg_z_text)
print("Number of sentences:", len(sentences))

# Function to create chunks of text
def chunk_sentences(sentences, max_chunk_size=1024):
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += ' ' + sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Create chunks from sentences
chunks = chunk_sentences(sentences)
print("Number of text chunks:", len(chunks))

# Summarize each chunk
summaries = []
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}")
    next_sum = summarizer(chunk, max_length=max_length, min_length=5, do_sample=False)
    summaries.append(next_sum)




Number of sentences: 80
Number of text chunks: 26
Processing chunk 1/26
Processing chunk 2/26
Processing chunk 3/26
Processing chunk 4/26
Processing chunk 5/26
Processing chunk 6/26
Processing chunk 7/26
Processing chunk 8/26
Processing chunk 9/26
Processing chunk 10/26
Processing chunk 11/26
Processing chunk 12/26
Processing chunk 13/26
Processing chunk 14/26
Processing chunk 15/26
Processing chunk 16/26
Processing chunk 17/26
Processing chunk 18/26
Processing chunk 19/26
Processing chunk 20/26
Processing chunk 21/26
Processing chunk 22/26
Processing chunk 23/26
Processing chunk 24/26
Processing chunk 25/26
Processing chunk 26/26


In [36]:
summaries

[[{'summary_text': 'Refinancing is a new transaction requiring new disclosures to the consumer. The new finance charge shall include any unearned portion of the old finance charge that is not credited to the existing obligation. The following shall not be treated as a refin'}],
 [{'summary_text': 'An assumption occurs when a creditor expressly agrees in writing with a subsequent consumer to accept that consumer as a primary obligor on an existing residential mortgage transaction. Before the assumption occurs, the creditor shall make new disclosures to the subsequent consumer, based'}],
 [{'summary_text': 'Rate adjustments with a corresponding change in payment. Creditor, assignee, or servicer of an adjustable-rate mortgage shall provide consumers with disclosures.'}],
 [{'summary_text': "An adjustable-rate mortgage or “ARM” is a closed-end consumer credit transaction secured by the consumer's principal dwelling in which the annual percentage rate may increase after consummation. The re

In [46]:
first_summary = ' '.join([summ[0]['summary_text'] for summ in summaries])
first_summary

"Refinancing is a new transaction requiring new disclosures to the consumer. The new finance charge shall include any unearned portion of the old finance charge that is not credited to the existing obligation. The following shall not be treated as a refin An assumption occurs when a creditor expressly agrees in writing with a subsequent consumer to accept that consumer as a primary obligor on an existing residential mortgage transaction. Before the assumption occurs, the creditor shall make new disclosures to the subsequent consumer, based Rate adjustments with a corresponding change in payment. Creditor, assignee, or servicer of an adjustable-rate mortgage shall provide consumers with disclosures. An adjustable-rate mortgage or “ARM” is a closed-end consumer credit transaction secured by the consumer's principal dwelling in which the annual percentage rate may increase after consummation. The requirements of thisparagraph (c) The disclosures required by thisparagraph (c)shall be provi

In [38]:
# Step 1: Combine the initial summaries
combined_summaries = " ".join([summ[0]['summary_text'] for summ in summaries])

# Split the combined summaries into sentences
sentences = sent_tokenize(combined_summaries)

# Create chunks
chunks = chunk_sentences(sentences)

print("Number of sentences:", len(sentences))
print("Number of text chunks:", len(chunks))

# Summarize each chunk
secondary_summaries = []
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}")
    next_sum = summarizer(chunk, max_length=max_length, min_length=5, do_sample=False)
    secondary_summaries.append(next_sum)



Number of sentences: 31
Number of text chunks: 7
Processing chunk 1/7
Processing chunk 2/7
Processing chunk 3/7
Processing chunk 4/7
Processing chunk 5/7
Processing chunk 6/7
Processing chunk 7/7


In [43]:

# Combine secondary summaries into a final summary
secondary_summary = ' '.join([summary[0]['summary_text'] for summary in secondary_summaries])
secondary_summary


"Refinancing is a new transaction requiring new disclosures to the consumer. The new finance charge shall include any unearned portion of the old finance charge that is not credited to the existing obligation. The disclosures required by thisparagraph (c)shall be provided to consumers at least 60, but no more than 120, days before the first payment at the adjusted level is due. The disclosures shall be provided for ARMs with uniformly scheduled The creditor, assignee, or servicer of an adjustable-rate mortgage shall provide consumers with disclosures in connection with the initial interest rate adjustment pursuant to the loan contract. The disclosures shall be provided to consumers at least 210, but no The amount of the current and new payment allocated to principal, interest, and taxes. The current payment allocation disclosed shall be the payment allocation for the last payment prior to the date of the disclosure. A statement that the new payment will not be allocated to pay loan pri

In [44]:
final_summary = summarizer(secondary_summary, do_sample=False)

In [45]:
final_summary[0]['summary_text']

'Refinancing is a new transaction requiring new disclosures to the consumer. The new finance charge shall include any unearned portion of the old finance charge. The amount of the current and new payment allocated to principal, interest, and taxes. A statement that the new payment will not be allocated to pay loan principal and will pay only part of the loan interest.'